In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier


In [3]:
#les données

results = pd.read_csv(r"data/results.csv", sep=',')
races = pd.read_csv(r"data/races.csv", sep=',')
circuits = pd.read_csv(r"data/circuits.csv", sep=',')
constructors = pd.read_csv(r"data/constructor_standings.csv", sep=',')

df1 = pd.merge(results[['driverId', 'constructorId', 'raceId', 'positionOrder']], races[['raceId', 'year', 'circuitId']], on = 'raceId')
df2 = pd.merge(df1, constructors[['raceId', 'constructorId', 'position', 'points']], on = ['raceId', 'constructorId'])
df3 = pd.merge(df2, circuits[['circuitId','country']], on = 'circuitId')
df3['winner'] = df3['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df4 = df3.drop(['circuitId', 'constructorId', 'positionOrder'], axis = 1)#.set_index(['raceId', 'driverId'])

label_encoder = LabelEncoder()
df4.loc[:, 'country'] = label_encoder.fit_transform(df3['country'])
df4.loc[:,"position**2"] = df4["position"] ** 2
df4.loc[:,"points**2"] = df4["points"] ** 2
df4 = df4.rename(columns={"points" : 'pointsequipe'})


In [4]:
print(df4.head())

   driverId  raceId  year  position  pointsequipe country  winner  \
0         1      18  2008         1          14.0       1       1   
1         2      18  2008         3           8.0       1       0   
2         3      18  2008         2           9.0       1       0   
3         4      18  2008         4           5.0       1       0   
4         5      18  2008         1          14.0       1       0   

   position**2  points**2  
0            1      196.0  
1            9       64.0  
2            4       81.0  
3           16       25.0  
4            1      196.0  


In [5]:
drivers = pd.read_csv(r"data\driver_standings.csv", sep=",") #on peut prendre le raceID, driverID, Points, Position, wins
driversINFO = pd.read_csv(r"data\drivers.csv",sep=",") #on prend la date de naisse dob[:4]
races = pd.read_csv(r"data\races.csv",sep=",") #raceID, year, circuitID, date, time
results = pd.read_csv(r"data\results.csv", sep=",") #driverID, raceID, positionORder, grid

results = results[['raceId','driverId','grid','positionOrder']] 
driversINFO = driversINFO[['driverId','dob']]
drivers = drivers[['raceId','driverId','points','wins']]
races = races[['raceId','year','circuitId','date','time']]

#transformation pour avoir la date de naissance des conducteurs
driversINFO.loc[:,'dob'] = pd.to_datetime(driversINFO['dob']).dt.year

#transformation de la date des courses pour avoir le mois 
races.loc[:,'date'] = pd.to_datetime(races['date']).dt.month
races = races.rename(columns={'date' : 'month'})

#retirer une erreur dans les dates et garder uniquement l'heure de la course
races["time"] = races['time'].replace({'\\N' : '12:00:00'})
races['hour'] = pd.to_datetime(races['time']).dt.hour
races = races.drop(columns = ["time"])

# on joint resultat et races sur la base de raceID
df12 = pd.merge(results[['raceId','driverId','grid','positionOrder']], races[['raceId','year','circuitId','month','hour']], on = 'raceId')

# on joint le précédent avec drivers avec drivers info sur driverID
df22 = pd.merge(drivers, driversINFO, on = 'driverId')

df32 = pd.merge(df12,df22, on = ['raceId', 'driverId'])
df32

#on utilise positionOrder pour faire le winner
df32['winner'] = df32['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df32["AgeAtRace"] = df32['year'] - df32['dob']
df42 = df32.drop(['positionOrder','circuitId', 'dob'], axis = 1)#.set_index(['raceId', 'driverId'])

#on retire qui a gagné la course à chaque course
df42["wins"] = df42["wins"] - df42["winner"]


C:\Users\luang\AppData\Local\Temp\ipykernel_4224\1924429326.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races['hour'] = pd.to_datetime(races['time']).dt.hour


In [6]:
print(df42.head())
print(df42.columns)

   raceId  driverId  grid  year month  hour  points  wins  winner AgeAtRace
0      18         1     1  2008     3     4    10.0     0       1        23
1      18         2     5  2008     3     4     8.0     0       0        31
2      18         3     7  2008     3     4     6.0     0       0        23
3      18         4    11  2008     3     4     5.0     0       0        27
4      18         5     3  2008     3     4     4.0     0       0        27
Index(['raceId', 'driverId', 'grid', 'year', 'month', 'hour', 'points', 'wins',
       'winner', 'AgeAtRace'],
      dtype='object')


In [7]:
DF = pd.merge(df4[['raceId','driverId', 'points**2',"position**2"]],df42[['raceId','driverId','year','month','hour','winner']], on=['raceId', 'driverId'])
DF = DF.set_index(['raceId','driverId'])
DF = DF.dropna(axis=1)
DF

points**2  position**2  year month  hour  winner
raceId driverId                                                  
18     1             196.0            1  2008     3     4       1
       2              64.0            9  2008     3     4       0
       3              81.0            4  2008     3     4       0
       4              25.0           16  2008     3     4       0
       5             196.0            1  2008     3     4       0
...                    ...          ...   ...   ...   ...     ...
1096   854          1369.0           64  2022    11    13       0
       825          1369.0           64  2022    11    13       0
       1          265225.0            9  2022    11    13       0
       849            64.0          100  2022    11    13       0
       4           29929.0           16  2022    11    13       0

[23716 rows x 6 columns]

In [70]:
DF.columns

Index(['points**2', 'position**2', 'year', 'month', 'hour', 'winner'], dtype='object')

In [71]:
DF_train = DF[DF['year']<=2020]
DF_test = DF[DF['year']==2021]

In [72]:
X_train_1 = DF_train.drop(columns="winner")
Y_train_1 = DF_train[['winner']]

X_test_1 = DF_test.drop(columns="winner")
Y_test_1 = DF_test[['winner']]

In [32]:
RFClassifier = RandomForestClassifier(random_state = 48)

param_grid = {
    'criterion' : ["gini"],
    'max_depth' : [30], 
    'min_samples_leaf' : [1],
    'n_estimators' : [50]

}

GRID = GridSearchCV(RFClassifier, param_grid,scoring="recall")
GRID.fit(X_train_1,Y_train_1.values.ravel())

print(GRID.best_params_)

{'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'n_estimators': 50}


{'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 1, 'n_estimators': 50} pour  environ 3,1secondes

In [73]:
GRID.score(X_train_1,Y_train_1.values.ravel())

0.2603305785123967

In [98]:
def fromPROBA_to_WinnerAndPodium(X_dateSet, Y_dataSet, Model):
    y_pred = Y_dataSet
    probabilite = Model.predict_proba(X_dateSet)
    probabilite = pd.DataFrame(probabilite)
    probabilite = probabilite.drop(columns=(0))
    probabilite = np.array(probabilite)
    y_pred.loc[:,'PREpositionOrder'] = probabilite
    
    Taille = y_pred.index
    raceID = Taille.get_level_values('raceId').unique()
    PodiumOr0 = []
    Top1 = []
    for RACE in raceID:
        driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
        array = np.array(y_pred["PREpositionOrder"][RACE])
        ranks = array.argsort()
        nb = len(driverID)
        ranks = nb - ranks.argsort()
        count = 0
        for DRIVER in driverID:
            rang = ranks[count]
            if rang <= 3:
                PodiumOr0.append(ranks[count])
                if rang == 1:
                    Top1.append(1)
                else:
                    Top1.append(0)
            else:
                PodiumOr0.append(0)
                Top1.append(0)
            count += 1 
        count = 0

    y_pred.loc[:,'winner'] = Top1
    y_pred.loc[:,'positionOrder'] = PodiumOr0
    fin = y_pred
    return fin


In [35]:
y_pred = Y_test_1
probabilite = GRID.predict_proba(X_train_1)
probabilite = pd.DataFrame(probabilite)
probabilite = probabilite.drop(columns=(0))
probabilite = np.array(probabilite)
y_pred.loc[:,'PREpositionOrder'] = probabilite

Taille = y_pred.index
raceID = Taille.get_level_values('raceId').unique()
PodiumOr0 = []
Top1 = []
for RACE in raceID:
    driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
    array = np.array(y_pred["PREpositionOrder"][RACE])
    ranks = array.argsort()
    nb = len(driverID)
    ranks = nb - ranks.argsort()
    count = 0
    for DRIVER in driverID:
        rang = ranks[count]
        if rang <= 3:
            PodiumOr0.append(ranks[count])
            if rang == 1:
                Top1.append(1)
            else:
                Top1.append(0)
        else:
            PodiumOr0.append(0)
            Top1.append(0)
        count += 1 
    count = 0

ValueError: Length of values (22836) does not match length of index (440)

In [75]:
y_preddata1 = fromPROBA_to_WinnerAndPodium(DF.drop(columns="winner"),DF[["winner"]])

C:\Users\luang\AppData\Local\Temp\ipykernel_4224\1054258265.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'PREpositionOrder'] = probabilite
C:\Users\luang\AppData\Local\Temp\ipykernel_4224\1054258265.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [78]:
#print(len(PodiumOr0),len(Top1))
y_preddata1

winner  PREpositionOrder  positionOrder
raceId driverId                                         
18     1              0          0.547000              2
       2              0          0.020000              3
       3              0          0.000000              0
       4              0          0.000000              0
       5              1          0.547000              1
...                 ...               ...            ...
1096   854            0          0.064333              0
       825            0          0.064333              0
       1              0          0.076667              0
       849            0          0.000000              0
       4              0          0.000000              0

[23716 rows x 3 columns]

In [ ]:
#y_pred.loc[:,'winner'] = Top1
#y_pred.loc[:,'positionOrder'] = PodiumOr0
#y_pred = y_pred.drop(columns=("PREpositionOrder"))
#y_pred

In [79]:
#y_pred.to_csv('MetaModel2/METAMODEL_RF_model_Constu.csv')
#y_preddata1.to_csv('MetaModel2/YALL_METAMODEL_RF_model_Constu.csv')

resultat de : 65.9

In [ ]:
#l'objectif est maintenant de prendre les probabilités, le winner et le classement de modèle en RF pour l'ajouter à un modèle Logistic
#Le logistic est sur les caractéristiques driver et les résultats du premier model

In [121]:
y_preddata1

winner  PREpositionOrder  positionOrder
raceId driverId                                         
18     1              0          0.547000              2
       2              0          0.020000              3
       3              0          0.000000              0
       4              0          0.000000              0
       5              1          0.547000              1
...                 ...               ...            ...
1096   854            0          0.064333              0
       825            0          0.064333              0
       1              0          0.076667              0
       849            0          0.000000              0
       4              0          0.000000              0

[23716 rows x 3 columns]

In [122]:
Ey_predRF = pd.read_csv(r'MetaModel2/YALL_METAMODEL_RF_model_Constu.csv', sep=',').rename(columns={'winner' : 'EwinnerRF', 'positionOrder' : 'EpositionOrderRF', 'PREpositionOrder' : 'EprobaRF'})
Ey_predRF

,raceId,driverId,EwinnerRF,EprobaRF,EpositionOrderRF
0,18,1,0,0.547000,2
1,18,2,0,0.020000,3
2,18,3,0,0.000000,0
3,18,4,0,0.000000,0
4,18,5,1,0.547000,1
...,...,...,...,...,...
23711,1096,854,0,0.064333,0
23712,1096,825,0,0.064333,0
23713,1096,1,0,0.076667,0
23714,1096,849,0,0.000000,0


In [123]:
Ey_predRF.columns

Index(['raceId', 'driverId', 'EwinnerRF', 'EprobaRF', 'EpositionOrderRF'], dtype='object')

In [124]:
DF2 = pd.merge(Ey_predRF[['raceId', 'driverId', 'EwinnerRF', 'EpositionOrderRF','EprobaRF']],
               df42[['raceId', 'driverId', 'grid', 'year', 'month', 'hour', 'points', 'wins','winner', 'AgeAtRace']],
               on = ['raceId', 'driverId'])

DF2 = DF2.set_index(['raceId','driverId'])
DF2 = DF2.dropna(axis=1)
DF2

EwinnerRF  EpositionOrderRF  EprobaRF  grid  year month  \
raceId driverId                                                            
18     1                 0                 2  0.547000     1  2008     3   
       2                 0                 3  0.020000     5  2008     3   
       3                 0                 0  0.000000     7  2008     3   
       4                 0                 0  0.000000    11  2008     3   
       5                 1                 1  0.547000     3  2008     3   
...                    ...               ...       ...   ...   ...   ...   
1096   854               0                 0  0.064333    12  2022    11   
       825               0                 0  0.064333    16  2022    11   
       1                 0                 0  0.076667     5  2022    11   
       849               0                 0  0.000000    20  2022    11   
       4                 0                 0  0.000000    10  2022    11   

                 hour  points  wins  winner AgeAtRace  
raceId driverId                                        
18     1            4    10.0     0       1        23  
       2            4     8.0     0       0        31  
       3            4     6.0     0       0        23  
       4            4     5.0     0       0        27  
       5            4     4.0     0       0        27  
...               ...     ...   ...     ...       ...  
1096   854         13    12.0     0       0        23  
       825         13    25.0     0       0        30  
       1           13   240.0     0       0        37  
       849         13     2.0     0       0        27  
       4           13    81.0     0       0        41  

[23750 rows x 11 columns]

In [125]:
DF2_train = DF2[DF2['year']<=2020]
DF2_test = DF2[DF2['year'] ==2021]

X_train_2 = DF2_train[['EwinnerRF', 'EpositionOrderRF','EprobaRF','grid','year','month','hour','points','wins','AgeAtRace']]
X_test_2 = DF2_test[['EwinnerRF', 'EpositionOrderRF','EprobaRF','grid','year','month','hour','points','wins','AgeAtRace']]

Y_train_2 = DF2_train[['winner']]
Y_test_2 = DF2_test[['winner']]

In [134]:
Logistic = LogisticRegression(random_state = 42)

# Définition de la grille de paramètres et du GridSearchCV
param_grid = {
    'penalty': [None],
    'max_iter': [4000],
    'solver' :["newton-cholesky"]
}

grid_search = GridSearchCV(Logistic, param_grid, scoring='recall')

# Entraînement du modèle
grid_search.fit(X_train_2, Y_train_2.values.ravel())
print(grid_search.best_params_)

grid_search = LogisticRegression(random_state = 42, max_iter= 4000, penalty = None, solver = 'newton-cholesky')
grid_search.fit(X_train_2, Y_train_2.values.ravel())

{'max_iter': 4000, 'penalty': None, 'solver': 'newton-cholesky'}


LogisticRegression(max_iter=4000, penalty=None, random_state=42,
                   solver='newton-cholesky')

best : {'max_iter': 4000, 'penalty': None, 'solver': 'newton-cholesky'}

In [127]:
grid_search.score(X_train_2,Y_train_2)

0.9833843463052033

In [128]:
X_train_2
Y_pred_METAMODEL2 = fromPROBA_to_WinnerAndPodium(X_test_2,Y_test_2, grid_search)

C:\Users\luang\AppData\Local\Temp\ipykernel_4224\4210917276.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'PREpositionOrder'] = probabilite
C:\Users\luang\AppData\Local\Temp\ipykernel_4224\4210917276.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [130]:
Y_pred_METAMODEL2
Y_pred_METAMODEL2.to_csv('MetaModel2\META_logistic_onRF_fromDriverConstruc.csv')

#résultats : 65.79